<a href="https://colab.research.google.com/github/saber-030/Python-Codes/blob/master/DNA_DataStorage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
#import statements
import lzma
import gzip
import bz2
import binascii
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd


In [2]:
def bzip_compression(data):
    data_out = bz2.compress(data)
    # print(len(data) / len(data_out))
 
    return data_out


In [3]:
def gzip_compression(data):
    data_out = gzip.compress(data)
    # print(len(data) / len(data_out))

    return data_out

In [4]:
def lzma_compression(data):
    data_out = lzma.compress(data)
    # print(len(data)/len(data_out))

    return data_out


In [5]:
def bzip_decompression(compressed_data):
    return bz2.decompress(compressed_data)

In [6]:
def gzip_decompression(compressed_data):
    return gzip.decompress(compressed_data)

In [7]:
def lzma_decompression(compressed_data):
    return lzma.decompress(compressed_data)

In [8]:
def data_base64(data):
    
    string=binascii.b2a_base64(data)

    return string

In [9]:
def data_reconvert(reconvert):
    return binascii.a2b_base64(reconvert)


In [10]:
# Creating tree nodes for Huffman Coding
class NodeTree(object):

    def __init__(self, left=None, right=None):
        self.left = left
        self.right = right

    def children(self):
        return (self.left, self.right)

    def nodes(self):
        return (self.left, self.right)

    def __str__(self):
        return '%s_%s' % (self.left, self.right)


# Main function implementing huffman coding
def huffman_code_tree(node, left=True, binString=''):
    if type(node) is str:
        return {node: binString}
    (l, r) = node.children()
    d = dict()
    d.update(huffman_code_tree(l, True, binString + '0'))
    d.update(huffman_code_tree(r, False, binString + '1'))
    return d


def Huffman_Compress(string, outGraph=False):
  # Calculating frequency
  freq = {}
  for c in string:
      if c in freq:
          freq[c] += 1
      else:
          freq[c] = 1

  freq = sorted(freq.items(), key=lambda x: x[1], reverse=True)

  nodes = freq

  while len(nodes) > 1:
      (key1, c1) = nodes[-1]
      (key2, c2) = nodes[-2]
      nodes = nodes[:-2]
      node = NodeTree(key1, key2)
      nodes.append((node, c1 + c2))

      nodes = sorted(nodes, key=lambda x: x[1], reverse=True)

  huffmanCode = huffman_code_tree(nodes[0][0])
  
  if outGraph == True:
    print(' Char | Huffman code ')
    print('----------------------')
    for (char, frequency) in freq:
        print(' %-4r |%12s' % (char, huffmanCode[char]))
  
  return huffmanCode

In [11]:
def toBinary(String, GraphKeys):
  Encoding = ""
  Sense = 'Even'
  for char in String:
    Encoding += (str(GraphKeys[char]))

  if len(Encoding)%2 != 0:
    Encoding += str(0)
    Sense = 'Odd'
  return Encoding, Sense

In [12]:
def toDNA(From_Binary_Encoding):
  DNA = ''
  (Binary_Encoding, Sense) = From_Binary_Encoding
  for char in range(0,len(Binary_Encoding),2):
    if (Binary_Encoding[char]=='0' and Binary_Encoding[char+1]=='0'):
      DNA += 'A'
    elif (Binary_Encoding[char]=='0' and Binary_Encoding[char+1]=='1'):
      DNA += 'T'
    elif (Binary_Encoding[char]=='1' and Binary_Encoding[char+1]=='0'):
      DNA += 'G'
    elif (Binary_Encoding[char]=='1' and Binary_Encoding[char+1]=='1'):
      DNA += 'C' 

  return DNA, Sense

In [13]:
!pip install cryptography
from cryptography.fernet import Fernet

def Encrypt(String):

  key = Fernet.generate_key()
  encryption_type = Fernet(key)
  String = str(encryption_type.encrypt(bytes(String, encoding='utf8')))

  return String

def Decrypt(String):
  String = encryption_type.decrypt(String)

  return String

     |████████████████████████████████| 2.7MB 2.7MB/s 


In [14]:
def Main_Compress(String, compressAlgo = 'bzip', encrypt = False, retainCase = False, outGraph = False):
  if retainCase == False:
    String = String.lower()

  if compressAlgo == 'bzip':
    String = bytes(String, 'utf-8')
    data_compress = data_base64(bzip_compression(String))
    String = str(data_compress, 'utf-8')
  elif compressAlgo == 'gzip':
    String = bytes(String, 'utf-8')
    data_compress = data_base64(gzip_compression(String))
    String = str(data_compress, 'utf-8')
  elif compressAlgo == 'lzma':
    String = bytes(String, 'utf-8')
    data_compress = data_base64(lzma_compression(String))
    String = str(data_compress, 'utf-8')
  elif compressAlgo == 'None':
    String = String            

  if encrypt == True:
    String = Encrypt(String)
  
  return(toDNA(toBinary(String, Huffman_Compress(String, outGraph))))

In [31]:
Test_String = """Lorem ipsum dolor sit amet, consectetur adipiscing elit. Morbi maximus nisi in nunc commodo, id malesuada odio vulputate. Vestibulum ante ipsum primis in faucibus orci luctus et ultrices posuere cubilia curae; Nam dapibus ac risus eu rutrum. Proin pretium eget nisl eu mattis. Nullam et tincidunt lorem, vitae vestibulum est. Nam luctus arcu a vehicula finibus. Sed cursus, est ut facilisis maximus, dolor augue dictum ipsum, quis fringilla nibh nibh a nisi. Curabitur lobortis elementum facilisis. Aliquam erat volutpat. Sed ultricies turpis sit amet dui mattis, ac aliquet purus efficitur. Nunc mi diam, interdum eu augue ut, tempus rhoncus ex. Etiam magna sapien, imperdiet et sem in, congue imperdiet mi. Sed venenatis magna purus, a faucibus neque tincidunt sed. Integer vel eleifend tellus. Lorem ipsum dolor sit amet, consectetur adipiscing elit. In hac habitasse platea dictumst. Quisque consectetur lorem vitae commodo consectetur. Mauris augue sem, convallis non ante sed, facilisis molestie nisi. Duis tempor sodales dignissim. Morbi aliquet orci non venenatis gravida. Ut eleifend eu purus eu euismod. Suspendisse interdum est eu dolor vulputate, eget rhoncus lacus placerat. Maecenas quis nulla vel tellus consequat pretium. Curabitur nibh nulla, aliquam at volutpat vitae, pretium at metus. Nunc elementum in neque eget rutrum. Etiam sit amet luctus magna. Proin pulvinar commodo libero, sed ultrices mauris mattis a. Vivamus tempus, libero non pretium laoreet, velit metus porta sapien, ac fermentum velit sapien et turpis. Sed tincidunt lacus sed magna porttitor tempor. Vestibulum in nibh at mauris tristique maximus. Ut commodo erat eu consequat luctus. Vivamus ligula libero, porttitor pharetra justo ut, sagittis luctus ante. Nulla rhoncus dictum vulputate. Cras euismod quam nec eros dapibus mollis. Nulla fermentum, nunc aliquam sollicitudin scelerisque, nulla justo elementum arcu, tempus tristique ante odio id magna. Curabitur consequat, quam eu ultrices molestie, sapien turpis viverra erat, vitae dapibus ipsum dui id enim. Sed id mauris magna. In interdum eu quam eget bibendum. Fusce sit amet urna ut mauris rutrum molestie ac in dui. Curabitur quis felis tortor. Suspendisse consectetur at diam sit amet pulvinar. Nunc eu elit eu dui euismod placerat. Fusce sodales tempor quam ac convallis. Proin sed ex libero. Donec mollis urna nec ipsum cursus, lobortis egestas risus eleifend. Integer viverra mollis nisi, vitae mollis diam pellentesque quis. Aenean sem quam, pellentesque sit amet porta nec, posuere et nunc. Duis tempor imperdiet massa, sit amet eleifend eros. Sed vulputate lectus non condimentum dapibus. Fusce scelerisque, odio sit amet eleifend varius, justo nisl mollis ipsum, id eleifend lectus justo a arcu. Aenean feugiat ex vitae libero sodales eleifend. Vestibulum nec ante sapien. Fusce eu ex tempus, imperdiet tortor nec, rhoncus dolor. Proin blandit consectetur dui et laoreet. Sed fermentum lectus sit amet facilisis ultricies. Aenean faucibus ex at tellus dignissim, ut bibendum elit pharetra. Vestibulum sit amet leo vitae risus iaculis consequat ac nec erat. Praesent ac sollicitudin lectus. Ut a fermentum nisl. Nunc in felis at tellus viverra convallis ac in dui. Pellentesque gravida, eros sed hendrerit auctor, libero urna sagittis eros, vitae imperdiet enim ex. Lorem ipsum dolor sit amet, consectetur adipiscing elit. Morbi maximus nisi in nunc commodo, id malesuada odio vulputate. Vestibulum ante ipsum primis in faucibus orci luctus et ultrices posuere cubilia curae; Nam dapibus ac risus eu rutrum. Proin pretium eget nisl eu mattis. Nullam et tincidunt lorem, vitae vestibulum est. Nam luctus arcu a vehicula finibus. Sed cursus, est ut facilisis maximus, dolor augue dictum ipsum, quis fringilla nibh nibh a nisi. Curabitur lobortis elementum facilisis. Aliquam erat volutpat. Sed ultricies turpis sit amet dui mattis, ac aliquet purus efficitur. Nunc mi diam, interdum eu augue ut, tempus rhoncus ex. Etiam magna sapien, imperdiet et sem in, congue imperdiet mi. Sed venenatis magna purus, a faucibus neque tincidunt sed. Integer vel eleifend tellus. Lorem ipsum dolor sit amet, consectetur adipiscing elit. In hac habitasse platea dictumst. Quisque consectetur lorem vitae commodo consectetur. Mauris augue sem, convallis non ante sed, facilisis molestie nisi. Duis tempor sodales dignissim. Morbi aliquet orci non venenatis gravida. Ut eleifend eu purus eu euismod. Suspendisse interdum est eu dolor vulputate, eget rhoncus lacus placerat. Maecenas quis nulla vel tellus consequat pretium. Curabitur nibh nulla, aliquam at volutpat vitae, pretium at metus. Nunc elementum in neque eget rutrum. Etiam sit amet luctus magna. Proin pulvinar commodo libero, sed ultrices mauris mattis a. Vivamus tempus, libero non pretium laoreet, velit metus porta sapien, ac fermentum velit sapien et turpis. Sed tincidunt lacus sed magna porttitor tempor. Vestibulum in nibh at mauris tristique maximus. Ut commodo erat eu consequat luctus. Vivamus ligula libero, porttitor pharetra justo ut, sagittis luctus ante. Nulla rhoncus dictum vulputate. Cras euismod quam nec eros dapibus mollis. Nulla fermentum, nunc aliquam sollicitudin scelerisque, nulla justo elementum arcu, tempus tristique ante odio id magna. Curabitur consequat, quam eu ultrices molestie, sapien turpis viverra erat, vitae dapibus ipsum dui id enim. Sed id mauris magna. In interdum eu quam eget bibendum. Fusce sit amet urna ut mauris rutrum molestie ac in dui. Curabitur quis felis tortor. Suspendisse consectetur at diam sit amet pulvinar. Nunc eu elit eu dui euismod placerat. Fusce sodales tempor quam ac convallis. Proin sed ex libero. Donec mollis urna nec ipsum cursus, lobortis egestas risus eleifend. Integer viverra mollis nisi, vitae mollis diam pellentesque quis. Aenean sem quam, pellentesque sit amet porta nec, posuere et nunc. Duis tempor imperdiet massa, sit amet eleifend eros. Sed vulputate lectus non condimentum dapibus. Fusce scelerisque, odio sit amet eleifend varius, justo nisl mollis ipsum, id eleifend lectus justo a arcu. Aenean feugiat ex vitae libero sodales eleifend. Vestibulum nec ante sapien. Fusce eu ex tempus, imperdiet tortor nec, rhoncus dolor. Proin blandit consectetur dui et laoreet. Sed fermentum lectus sit amet facilisis ultricies. Aenean faucibus ex at tellus dignissim, ut bibendum elit pharetra. Vestibulum sit amet leo vitae risus iaculis consequat ac nec erat. Praesent ac sollicitudin lectus. Ut a fermentum nisl. Nunc in felis at tellus viverra convallis ac in dui. Pellentesque gravida, eros sed hendrerit auctor, libero urna sagittis eros, vitae imperdiet enim ex."""
file=open("/bible.txt", "r")
file_list=["/alice29.txt","/E.coli","/asyoulik.txt","/bible.txt","/lcet10.txt","/plrabn12.txt","/world192.txt"]
Test_String=file.read()
z=Main_Compress(Test_String)
res = ''.join(format(ord(i), 'b') for i in Test_String)
len_res = len(str(res))/2 
(DNA_Codes, Sense) = Main_Compress(Test_String)
l = len_res/len(DNA_Codes)
print(l)

4.0425067874239415
4.0425067874239415


In [32]:
res = ''.join(format(ord(i), 'b') for i in Test_String)
len_res = len(str(res))/2 
for x in ['bzip','gzip','lzma','None']:
  for y in [True, False]:
    for z in [True, False]:
      file=open("/bible.txt","r")
      Test_String=file.read()
      (DNA_Codes, Sense) = Main_Compress(Test_String, x, y, z)
      l = len_res/len(DNA_Codes)
      print("Compress Algo: {} Encryption: {:5s} CaseRetain: {:5s} :: Compress Ratio = {:1.3f}".format(x,str(y),str(z),l))

Compress Algo: bzip Encryption: True  CaseRetain: True  :: Compress Ratio = 3.024
Compress Algo: bzip Encryption: True  CaseRetain: False :: Compress Ratio = 3.030
Compress Algo: bzip Encryption: False CaseRetain: True  :: Compress Ratio = 4.034
Compress Algo: bzip Encryption: False CaseRetain: False :: Compress Ratio = 4.043
Compress Algo: gzip Encryption: True  CaseRetain: True  :: Compress Ratio = 2.172
Compress Algo: gzip Encryption: True  CaseRetain: False :: Compress Ratio = 2.207
Compress Algo: gzip Encryption: False CaseRetain: True  :: Compress Ratio = 2.897
Compress Algo: gzip Encryption: False CaseRetain: False :: Compress Ratio = 2.942
Compress Algo: lzma Encryption: True  CaseRetain: True  :: Compress Ratio = 2.889
Compress Algo: lzma Encryption: True  CaseRetain: False :: Compress Ratio = 2.923
Compress Algo: lzma Encryption: False CaseRetain: True  :: Compress Ratio = 3.853
Compress Algo: lzma Encryption: False CaseRetain: False :: Compress Ratio = 3.897
Compress Algo: N

In [35]:
res = ''.join(format(ord(i), 'b') for i in Test_String)
len_res = len(str(res))/2 
for x in ['bzip','gzip','lzma','None']:
  for y in [True, False]:
    for z in [True, False]:
      file=open("/alice29.txt","r")
      Test_String=file.read()
      (DNA_Codes, Sense) = Main_Compress(Test_String, x, y, z)
      l = len_res/len(DNA_Codes)
      print("Compress Algo: {} Encryption: {:5s} CaseRetain: {:5s} :: Compress Ratio = {:1.3f}".format(x,str(y),str(z),l))

Compress Algo: bzip Encryption: True  CaseRetain: True  :: Compress Ratio = 2.150
Compress Algo: bzip Encryption: True  CaseRetain: False :: Compress Ratio = 2.216
Compress Algo: bzip Encryption: False CaseRetain: True  :: Compress Ratio = 2.872
Compress Algo: bzip Encryption: False CaseRetain: False :: Compress Ratio = 2.961
Compress Algo: gzip Encryption: True  CaseRetain: True  :: Compress Ratio = 1.736
Compress Algo: gzip Encryption: True  CaseRetain: False :: Compress Ratio = 1.793
Compress Algo: gzip Encryption: False CaseRetain: True  :: Compress Ratio = 2.317
Compress Algo: gzip Encryption: False CaseRetain: False :: Compress Ratio = 2.393
Compress Algo: lzma Encryption: True  CaseRetain: True  :: Compress Ratio = 1.937
Compress Algo: lzma Encryption: True  CaseRetain: False :: Compress Ratio = 1.996
Compress Algo: lzma Encryption: False CaseRetain: True  :: Compress Ratio = 2.585
Compress Algo: lzma Encryption: False CaseRetain: False :: Compress Ratio = 2.664
Compress Algo: N

In [37]:
res = ''.join(format(ord(i), 'b') for i in Test_String)
len_res = len(str(res))/2 
for x in ['bzip','gzip','lzma','None']:
  for y in [True, False]:
    for z in [True, False]:
      file=open("/E.coli","r")
      Test_String=file.read()
      (DNA_Codes, Sense) = Main_Compress(Test_String, x, y, z)
      l = len_res/len(DNA_Codes)
      print("Compress Algo: {} Encryption: {:5s} CaseRetain: {:5s} :: Compress Ratio = {:1.3f}".format(x,str(y),str(z),l))

Compress Algo: bzip Encryption: True  CaseRetain: True  :: Compress Ratio = 2.427
Compress Algo: bzip Encryption: True  CaseRetain: False :: Compress Ratio = 2.427
Compress Algo: bzip Encryption: False CaseRetain: True  :: Compress Ratio = 3.257
Compress Algo: bzip Encryption: False CaseRetain: False :: Compress Ratio = 3.257
Compress Algo: gzip Encryption: True  CaseRetain: True  :: Compress Ratio = 2.336
Compress Algo: gzip Encryption: True  CaseRetain: False :: Compress Ratio = 2.336
Compress Algo: gzip Encryption: False CaseRetain: True  :: Compress Ratio = 3.116
Compress Algo: gzip Encryption: False CaseRetain: False :: Compress Ratio = 3.116
Compress Algo: lzma Encryption: True  CaseRetain: True  :: Compress Ratio = 2.559
Compress Algo: lzma Encryption: True  CaseRetain: False :: Compress Ratio = 2.559
Compress Algo: lzma Encryption: False CaseRetain: True  :: Compress Ratio = 3.412
Compress Algo: lzma Encryption: False CaseRetain: False :: Compress Ratio = 3.412
Compress Algo: N

In [39]:
res = ''.join(format(ord(i), 'b') for i in Test_String)
len_res = len(str(res))/2 
for x in ['bzip','gzip','lzma','None']:
  for y in [True, False]:
    for z in [True, False]:
      file=open("/asyoulik.txt","r")
      Test_String=file.read()
      (DNA_Codes, Sense) = Main_Compress(Test_String, x, y, z)
      l = len_res/len(DNA_Codes)
      print("Compress Algo: {} Encryption: {:5s} CaseRetain: {:5s} :: Compress Ratio = {:1.3f}".format(x,str(y),str(z),l))

Compress Algo: bzip Encryption: True  CaseRetain: True  :: Compress Ratio = 1.962
Compress Algo: bzip Encryption: True  CaseRetain: False :: Compress Ratio = 1.991
Compress Algo: bzip Encryption: False CaseRetain: True  :: Compress Ratio = 2.620
Compress Algo: bzip Encryption: False CaseRetain: False :: Compress Ratio = 2.659
Compress Algo: gzip Encryption: True  CaseRetain: True  :: Compress Ratio = 1.591
Compress Algo: gzip Encryption: True  CaseRetain: False :: Compress Ratio = 1.639
Compress Algo: gzip Encryption: False CaseRetain: True  :: Compress Ratio = 2.124
Compress Algo: gzip Encryption: False CaseRetain: False :: Compress Ratio = 2.188
Compress Algo: lzma Encryption: True  CaseRetain: True  :: Compress Ratio = 1.743
Compress Algo: lzma Encryption: True  CaseRetain: False :: Compress Ratio = 1.785
Compress Algo: lzma Encryption: False CaseRetain: True  :: Compress Ratio = 2.327
Compress Algo: lzma Encryption: False CaseRetain: False :: Compress Ratio = 2.384
Compress Algo: N

In [42]:
res = ''.join(format(ord(i), 'b') for i in Test_String)
len_res = len(str(res))/2 
for x in ['bzip','gzip','lzma','None']:
  for y in [True, False]:
    for z in [True, False]:
      file=open("/lcet10.txt","r")
      Test_String=file.read()
      (DNA_Codes, Sense) = Main_Compress(Test_String, x, y, z)
      l = len_res/len(DNA_Codes)
      print("Compress Algo: {} Encryption: {:5s} CaseRetain: {:5s} :: Compress Ratio = {:1.3f}".format(x,str(y),str(z),l))

Compress Algo: bzip Encryption: True  CaseRetain: True  :: Compress Ratio = 2.452
Compress Algo: bzip Encryption: True  CaseRetain: False :: Compress Ratio = 2.517
Compress Algo: bzip Encryption: False CaseRetain: True  :: Compress Ratio = 3.272
Compress Algo: bzip Encryption: False CaseRetain: False :: Compress Ratio = 3.358
Compress Algo: gzip Encryption: True  CaseRetain: True  :: Compress Ratio = 1.851
Compress Algo: gzip Encryption: True  CaseRetain: False :: Compress Ratio = 1.910
Compress Algo: gzip Encryption: False CaseRetain: True  :: Compress Ratio = 2.470
Compress Algo: gzip Encryption: False CaseRetain: False :: Compress Ratio = 2.548
Compress Algo: lzma Encryption: True  CaseRetain: True  :: Compress Ratio = 2.236
Compress Algo: lzma Encryption: True  CaseRetain: False :: Compress Ratio = 2.294
Compress Algo: lzma Encryption: False CaseRetain: True  :: Compress Ratio = 2.983
Compress Algo: lzma Encryption: False CaseRetain: False :: Compress Ratio = 3.060
Compress Algo: N

In [44]:
res = ''.join(format(ord(i), 'b') for i in Test_String)
len_res = len(str(res))/2 
for x in ['bzip','gzip','lzma','None']:
  for y in [True, False]:
    for z in [True, False]:
      file=open("/plrabn12.txt","r")
      Test_String=file.read()
      (DNA_Codes, Sense) = Main_Compress(Test_String, x, y, z)
      l = len_res/len(DNA_Codes)
      print("Compress Algo: {} Encryption: {:5s} CaseRetain: {:5s} :: Compress Ratio = {:1.3f}".format(x,str(y),str(z),l))

Compress Algo: bzip Encryption: True  CaseRetain: True  :: Compress Ratio = 2.035
Compress Algo: bzip Encryption: True  CaseRetain: False :: Compress Ratio = 2.069
Compress Algo: bzip Encryption: False CaseRetain: True  :: Compress Ratio = 2.715
Compress Algo: bzip Encryption: False CaseRetain: False :: Compress Ratio = 2.761
Compress Algo: gzip Encryption: True  CaseRetain: True  :: Compress Ratio = 1.533
Compress Algo: gzip Encryption: True  CaseRetain: False :: Compress Ratio = 1.578
Compress Algo: gzip Encryption: False CaseRetain: True  :: Compress Ratio = 2.045
Compress Algo: gzip Encryption: False CaseRetain: False :: Compress Ratio = 2.104
Compress Algo: lzma Encryption: True  CaseRetain: True  :: Compress Ratio = 1.797
Compress Algo: lzma Encryption: True  CaseRetain: False :: Compress Ratio = 1.840
Compress Algo: lzma Encryption: False CaseRetain: True  :: Compress Ratio = 2.396
Compress Algo: lzma Encryption: False CaseRetain: False :: Compress Ratio = 2.454
Compress Algo: N

In [46]:
res = ''.join(format(ord(i), 'b') for i in Test_String)
len_res = len(str(res))/2 
for x in ['bzip','gzip','lzma','None']:
  for y in [True, False]:
    for z in [True, False]:
      file=open("/world192.txt","r")
      Test_String=file.read()
      (DNA_Codes, Sense) = Main_Compress(Test_String, x, y, z)
      l = len_res/len(DNA_Codes)
      print("Compress Algo: {} Encryption: {:5s} CaseRetain: {:5s} :: Compress Ratio = {:1.3f}".format(x,str(y),str(z),l))

Compress Algo: bzip Encryption: True  CaseRetain: True  :: Compress Ratio = 3.047
Compress Algo: bzip Encryption: True  CaseRetain: False :: Compress Ratio = 3.069
Compress Algo: bzip Encryption: False CaseRetain: True  :: Compress Ratio = 4.065
Compress Algo: bzip Encryption: False CaseRetain: False :: Compress Ratio = 4.093
Compress Algo: gzip Encryption: True  CaseRetain: True  :: Compress Ratio = 2.082
Compress Algo: gzip Encryption: True  CaseRetain: False :: Compress Ratio = 2.156
Compress Algo: gzip Encryption: False CaseRetain: True  :: Compress Ratio = 2.777
Compress Algo: gzip Encryption: False CaseRetain: False :: Compress Ratio = 2.874
Compress Algo: lzma Encryption: True  CaseRetain: True  :: Compress Ratio = 3.076
Compress Algo: lzma Encryption: True  CaseRetain: False :: Compress Ratio = 3.134
Compress Algo: lzma Encryption: False CaseRetain: True  :: Compress Ratio = 4.102
Compress Algo: lzma Encryption: False CaseRetain: False :: Compress Ratio = 4.179
Compress Algo: N

In [53]:
res = ''.join(format(ord(i), 'b') for i in Test_String)
len_res = len(str(res))/2 
for x in ['bzip','gzip','lzma','None']:
  for y in [True, False]:
    for z in [True, False]:
      file=open("/fields.c","r")
      Test_String=file.read()
      (DNA_Codes, Sense) = Main_Compress(Test_String, x, y, z)
      l = len_res/len(DNA_Codes)
      print("Compress Algo: {} Encryption: {:5s} CaseRetain: {:5s} :: Compress Ratio = {:1.3f}".format(x,str(y),str(z),l))

Compress Algo: bzip Encryption: True  CaseRetain: True  :: Compress Ratio = 2.145
Compress Algo: bzip Encryption: True  CaseRetain: False :: Compress Ratio = 2.242
Compress Algo: bzip Encryption: False CaseRetain: True  :: Compress Ratio = 2.918
Compress Algo: bzip Encryption: False CaseRetain: False :: Compress Ratio = 3.041
Compress Algo: gzip Encryption: True  CaseRetain: True  :: Compress Ratio = 2.089
Compress Algo: gzip Encryption: True  CaseRetain: False :: Compress Ratio = 2.163
Compress Algo: gzip Encryption: False CaseRetain: True  :: Compress Ratio = 2.827
Compress Algo: gzip Encryption: False CaseRetain: False :: Compress Ratio = 2.931
Compress Algo: lzma Encryption: True  CaseRetain: True  :: Compress Ratio = 2.152
Compress Algo: lzma Encryption: True  CaseRetain: False :: Compress Ratio = 2.204
Compress Algo: lzma Encryption: False CaseRetain: True  :: Compress Ratio = 2.919
Compress Algo: lzma Encryption: False CaseRetain: False :: Compress Ratio = 2.997
Compress Algo: N

In [56]:
res = ''.join(format(ord(i), 'b') for i in Test_String)
len_res = len(str(res))/2 
for x in ['bzip','gzip','lzma','None']:
  for y in [True, False]:
    for z in [True, False]:
      file=open("/grammar.lsp","r")
      Test_String=file.read()
      (DNA_Codes, Sense) = Main_Compress(Test_String, x, y, z)
      l = len_res/len(DNA_Codes)
      print("Compress Algo: {} Encryption: {:5s} CaseRetain: {:5s} :: Compress Ratio = {:1.3f}".format(x,str(y),str(z),l))

Compress Algo: bzip Encryption: True  CaseRetain: True  :: Compress Ratio = 1.688
Compress Algo: bzip Encryption: True  CaseRetain: False :: Compress Ratio = 1.764
Compress Algo: bzip Encryption: False CaseRetain: True  :: Compress Ratio = 2.359
Compress Algo: bzip Encryption: False CaseRetain: False :: Compress Ratio = 2.460
Compress Algo: gzip Encryption: True  CaseRetain: True  :: Compress Ratio = 1.748
Compress Algo: gzip Encryption: True  CaseRetain: False :: Compress Ratio = 1.815
Compress Algo: gzip Encryption: False CaseRetain: True  :: Compress Ratio = 2.439
Compress Algo: gzip Encryption: False CaseRetain: False :: Compress Ratio = 2.526
Compress Algo: lzma Encryption: True  CaseRetain: True  :: Compress Ratio = 1.686
Compress Algo: lzma Encryption: True  CaseRetain: False :: Compress Ratio = 1.715
Compress Algo: lzma Encryption: False CaseRetain: True  :: Compress Ratio = 2.334
Compress Algo: lzma Encryption: False CaseRetain: False :: Compress Ratio = 2.397
Compress Algo: N

In [62]:
res = ''.join(format(ord(i), 'b') for i in Test_String)
len_res = len(str(res))/2 
for x in ['bzip','gzip','lzma','None']:
  for y in [True, False]:
    for z in [True, False]:
      file=open("/xargs.1","r")
      Test_String=file.read()
      (DNA_Codes, Sense) = Main_Compress(Test_String, x, y, z)
      l = len_res/len(DNA_Codes)
      print("Compress Algo: {} Encryption: {:5s} CaseRetain: {:5s} :: Compress Ratio = {:1.3f}".format(x,str(y),str(z),l))

Compress Algo: bzip Encryption: True  CaseRetain: True  :: Compress Ratio = 1.455
Compress Algo: bzip Encryption: True  CaseRetain: False :: Compress Ratio = 1.527
Compress Algo: bzip Encryption: False CaseRetain: True  :: Compress Ratio = 2.008
Compress Algo: bzip Encryption: False CaseRetain: False :: Compress Ratio = 2.110
Compress Algo: gzip Encryption: True  CaseRetain: True  :: Compress Ratio = 1.475
Compress Algo: gzip Encryption: True  CaseRetain: False :: Compress Ratio = 1.527
Compress Algo: gzip Encryption: False CaseRetain: True  :: Compress Ratio = 2.021
Compress Algo: gzip Encryption: False CaseRetain: False :: Compress Ratio = 2.103
Compress Algo: lzma Encryption: True  CaseRetain: True  :: Compress Ratio = 1.418
Compress Algo: lzma Encryption: True  CaseRetain: False :: Compress Ratio = 1.466
Compress Algo: lzma Encryption: False CaseRetain: True  :: Compress Ratio = 1.953
Compress Algo: lzma Encryption: False CaseRetain: False :: Compress Ratio = 2.007
Compress Algo: N